# Sistemas Inteligentes

## Curso académico 2024-2025

### Laboratorio 2: Búsqueda Metaheurística

#### Instructores

* Juan Carlos Alfaro Jiménez: JuanCarlos.Alfaro@uclm.es
* María Julia Flores Gallego: Julia.Flores@uclm.es
* Ismael García Varea: Ismael.Garcia@uclm.es
* Adrián Rodríguez López: Adrian.Rodriguez18@alu.uclm.es

## Estaciones de Servicio y Energía: Encontrando la Configuración Óptima

## 1. Introducción

¡Noticias emocionantes! El **Ministerio de Transporte y Movilidad Sostenible** ha quedado muy impresionado con las soluciones desarrolladas en nuestro primer trabajo, la Práctica 1. Están particularmente interesados en implementar estos algoritmos en la planificación de rutas de vehículos autónomos, con A* como el método MÁS efectivo para identificar el camino óptimo de manera eficiente. Para avanzar en este proyecto, el Ministerio tiene como objetivo establecer estratégicamente estaciones de servicio en áreas urbanas para apoyar su flota de vehículos autónomos. Estas estaciones funcionarán como centros de flota y proporcionarán servicios esenciales para los vehículos.

Para lograr esto, han solicitado **nuestra experiencia técnica para determinar la distribución óptima de estas estaciones** en los mapas de la ciudad. Sin embargo, no todas las intersecciones son elegibles como ubicación de una estación; el Ministerio ha preseleccionado intersecciones candidatas basándose en criterios específicos establecidos por sus equipos técnicos y administrativos. Su principal enfoque es la sostenibilidad y el acceso equitativo, con el objetivo de garantizar que todos los ciudadanos estén razonablemente cerca de una estación de servicio. Entre estos puntos seleccionados, solo se elegirá un número específico. Para facilitar nuestra tarea de determinar cuáles deben ser, han proporcionado datos sobre la cobertura poblacional de cada intersección candidata, lo que nos permite tener en cuenta tanto el acceso como la cobertura en nuestra estrategia de distribución.

El objetivo principal es garantizar un acceso eficiente a la máxima población posible, manteniendo una distribución equilibrada en toda la ciudad, una consideración vital para un sistema de transporte completamente autónomo.

### 1.1 Objetivos del Laboratorio

En esta práctica, aplicaremos técnicas de búsqueda metaheurística para resolver problemas de optimización combinatoria.

El primer objetivo es comprender la tarea y formularla desde la perspectiva de la búsqueda metaheurística. Implementaremos al menos dos algoritmos:

* **Búsqueda Aleatoria**, como un punto de partida básico que generará múltiples soluciones, evaluará cada una y devolverá la mejor.

* **Algoritmo Genético**, que permitirá configurar varios parámetros, como el tamaño de la población, la tasa de mutación y el número de generaciones, entre otros.


Además, para la evaluación no-continua se tendrá que implementar la Ascensión de Cplinas o Hill Cilimbing (obligatoriamente), y opcionalmente implementar el algoritmo **Iterated Local Search** (ILS), ambos explicados en el Tema 7.

A continuación, analizaremos y compararemos el rendimiento de estos algoritmos ejecutándolos en instancias de problemas de diferente complejidad.

Esperamos que esta práctica te ayude a profundizar en tu comprensión de las técnicas metaheurísticas y te anime a considerar cómo se pueden aplicar en problemas reales de optimización combinatoria.

**¡Buena suerte!**

## 2. Descripción del Problema

### 2.1 Problemas de Entrada

Cada escenario se proporcionará en un archivo en formato `json` que contiene la siguiente información, con el formato de un diccionario cuyas claves son:

* `address`: La dirección utilizada.
* `distance`: Radio máximo utilizado para definir intersecciones y segmentos alrededor de la dirección.
* `intersections`: Una lista de diccionarios con información sobre las intersecciones.
* `segments`: Una lista de diccionarios con información sobre los segmentos, que representan las calles entre dos intersecciones.
* `candidates`: Una lista de pares (identificador, población) que contiene las intersecciones candidatas. Notad que los identificadores en esta lista deben estar incluidos en la lista de intersecciones.
* `number_stations`: El número de estaciones de servicio que se deben ubicar, que no debe superar el número de candidatos.

Cada diccionario en `intersections` incluye tres claves:

* `identifier`: Identificador de la intersección
* `longitude`: Longitud de la intersección
* `latitude`: Latitud de la intersección

Cada diccionario en `segments` incluye cuatro claves:

* `origin`: Intersección de origen
* `destination`: Intersección de destino
* `distance`: Distancia entre las dos intersecciones
* `speed`: Velocidad máxima permitida entre las dos intersecciones

**IMPORTANTE**: `initial` y `final` ya no están incluidos en el archivo JSON, ya que no son necesarios. Durante la evaluación de una posible configuración, estos puntos iniciales y finales cambiarán varias veces. Esto puede requerir algunos ajustes en el código de la Práctica 1 para ejecutar A*. Estos cambios deben estar claramente indicados (tu código debe coincidir con el de la Práctica 1, excepto por estos cambios) y discutidos en la memoria de prácticas.

### 2.2. Ejemplo ilustrativo

Un posible ejemplo de este problema podría ser el que se muestra en la siguiente imagen, que representa una parte de la ciudad de Albacete:

![title](p2/sample-problems-lab2/toy/example.png)

En este caso, el número de estaciones de servicio de vehículos que se deben ubicar es 4, entre las 15 intersecciones candidatas representadas con puntos azules (etiquetadas con la población cubierta). Una posible solución se representa con puntos verdes.

---

##### Nota:

* El archivo que contiene la imagen debe guardarse en la ruta indicada en el código de esta celda.

---

### 2.3 Definición formal del problema

Necesitamos elegir $s$ estaciones de entre $c$ intersecciones candidatas o elegibles, con $s<c$. Por lo tanto, nuestro objetivo es decidir en cuál de estas $c$ intersecciones candidatas debemos ubicar las $s$ estaciones de servicio de vehículos, de manera que se minimice el tiempo promedio de viaje que cada habitante tarda desde su hogar hasta la estación más cercana. Si denotamos por $S$ al vector de tamaño $s$ que contiene las intersecciones en las que se ubican las estaciones de vehículos y por $C$ al vector de intersecciones candidatas que contiene el par (id, pop) para cada intersección candidata, entonces formalmente, queremos resolver el siguiente problema de optimización:

$$
S^* = \arg\min_{S} \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \sum_{i=0}^{c-1} \; C[i].pop \cdot \min_{j=0,\dots,s-1} \big\{time(C[i].id,S[j])\big\}
$$

donde:
- $C[i].pop$ representa la población (número de habitantes) cubierta por la intersección candidata $i$.
- $C[i].id$ es el identificador de la intersección candidata $i$.
- $time(A,B)$ representa el menor tiempo real para viajar desde la intersección $A$ hasta la intersección $B$.
- $S$ contiene exactamente $s$ intersecciones distintas, que deben pertenecer todas al conjunto de soluciones candidatas, y siendo $s<=c$

Las siguientes consideraciones deben tenerse en cuenta respecto a la expresión anterior:
- Estamos tratando con un problema de minimización.
- La cardinalidad del espacio de búsqueda es:

$$
\binom{c}{s} = \frac{c!}{(c-s)!s!}
$$

por ejemplo, si tenemos 20 intersecciones elegibles y 4 estaciones de vehículos, el número de soluciones posibles es 210, no demasiadas; pero si tenemos 100 candidatos y 10 estaciones, entonces el número de soluciones posibles es $1.7\times10^{13}$ ($5.3\times 10^{20}$ con 20 estaciones).

## 3. Desarrollo de la práctica

Antes de implementar los algoritmos, primero debes considerar definir los elementos básicos en este tipo de problemas, a saber:

- Una representación conveniente para las soluciones (configuraciones, cromosomas, individuos, ...) del problema que se utilizarán en los algoritmos de optimización combinatoria. Piensa detenidamente en las distintas opciones y tendrás que discutirlas en el informe de la tarea.

- Implementar un mecanismo de evaluación para gestionar las evaluaciones realizadas por los algoritmos de optimización combinatoria. A continuación, se detallará cómo debe realizarse la evaluación.

- Notas importantes:
    - En el caso de que A* no devuelva ninguna solución (coste = inf), reemplazad este valor por un número muy alto en comparación con el tiempo máximo en nuestro problema. Reflexiona sobre la necesidad de esto y discútelo en el informe.
    - Podéis aprovechar el mecanismo de evaluación para guardar algunos cálculos, recopilar estadísticas e imprimir los resultados.
    - Tened en cuenta que esta tarea requiere que ya hayas resuelto la Práctica 1, y necesitarás reutilizar el código implementado para resolver esta práctica.
   
Tendréis que resolver muchos problemas similares a los de la Práctica 1. Los mapas serán los mismos, pero los problemas necesitan incorporar nueva información, que es la lista de intersecciones candidatas y, para cada una de ellas, la población que cubren. El número de estaciones que se deben ubicar también se indica en el problema como `number_stations`.


### 3.1 Evaluación de una solución

Dada una instancia específica del problema a resolver, y asumiendo que $C$ denota su lista de intersecciones candidatas, el valor de cada posible solución $S$ debe calcularse como:

$$value(S) = \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \sum_{i=0}^{c-1} \; C[i].pop \cdot \min_{j=0,\dots,s-1} \big\{ time(C[i].id,S[j])\big\}$$

de acuerdo con la fórmula presentada en la sección 2.2.

## 4. Plan de trabajo

### 4.1. Tareas

* Transferid y adaptad vuestro código de la Práctica 1 para resolver búsquedas con A* que necesitaréis aquí:
    * Reutilizad la mayor parte del código necesario de vuestra Práctica 1.
    * Describid qué se ha modificado, por qué y cómo.

* Procesad los nuevos archivos JSON y guardad el problema de acuerdo con lo siguiente:
    * Además de las clases de búsqueda (Problem_2, State, Action, ...), deberéis trabajar con las intersecciones candidatas.
    * Construid un mecanismo capaz de almacenar y recuperar las intersecciones candidatas y la población asociada a cada una.

* Representación de una posible configuración:
    * Entre las vistas en el Tema 6, encontrad la representación más adecuada para este problema y adaptadla, teniendo en cuenta que cada problema tiene valores distintos para el número total de intersecciones, intersecciones elegibles y el número solicitado de estaciones.
    * Conectadla a un método adecuado para evaluar cada una considerando las indicaciones mencionadas anteriormente en el punto 3.2.

* Implementación de algoritmos:
    * Implementad, al menos, los dos algoritmos requeridos (Búsqueda Aleatoria y un Algoritmo Genético — GA). Tened en cuenta que para la evaluación no continua, también deberéis agregar Hill Climbing y, opcionalmente, ILS.
    * En el GA, aseguraos de haber implementado la generación de una población junto con los elementos principales dentro del bucle principal: selección, cruce, mutación y combinación de generaciones.

* Experimentación y análisis:
    * Los parámetros que se puedan ajustar deben ser explorados adecuadamente, también en relación con los problemas dados (dimensionalidad, complejidad, etc.).
    * También deberéis estudiar el rendimiento resultante en términos de desempeño, convergencia, número de generaciones, etc.
    * Comparad la Búsqueda Aleatoria y el Algoritmo Genético, asegurándoos de obtener resultados consistentes.

* Informe:
    * Redactad un informe detallando el proceso seguido, las estrategias implementadas y los resultados obtenidos, junto con gráficos y comparaciones visuales.


### 4.2. Evaluación de la práctica

En la modalidad de **evaluación continua**, la evaluación de la práctica se realizará a través de un examen individual en el que se tendrá en cuenta lo siguiente:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 25%
* Implementación correcta del algoritmo genético: 50%, que cubre
    * El bucle general para las generaciones es correcto: 10%
    * Los distintos operadores están correctamente diseñados y codificados: 40%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%

Es necesario que la Práctica 1 esté correctamente integrada y que la Búsqueda Aleatoria funcione de manera consistente para que todos los estudiantes puedan utilizarla como un punto de partida base adecuado.

Todo esto se ponderará según el nivel de conocimiento que el estudiante demuestre sobre la práctica en caso de que el examen sea una entrevista personal.

En la modalidad de **evaluación no continua**, la evaluación se modificará como se indica a continuación:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 15%
* Implementación correcta del algoritmo genético: 40%, que cubre
    * El bucle general para las generaciones es correcto: 7%
    * Los distintos operadores están correctamente diseñados y codificados: 33%
* Implementación correcta del algoritmo de Hill Climbing (obligatorio): 15%
* Implementación correcta del algoritmo ILS (opcional): 5%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%


### 4.3. Fechas importantes

* Fecha límite para entregar el código: **13 de diciembre de 2024**.
* Fecha límite para la entrega del informe: **Final del semestre**.


In [10]:
import json
import math
import random
import sys
import time
import heapq
import statistics
from collections import defaultdict


class Problema:
    def __init__(self, nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            problema = json.load(archivo)

        self.inicio = None
        self.final = None
        self.interseccionAccion = defaultdict(list)
        self.interseccionesCoordenadas = {}
        self.velMax = -1

        for interseccion in problema['intersections']:
            identificador = interseccion['identifier']
            self.interseccionesCoordenadas[identificador] = (interseccion['longitude'], interseccion['latitude'])

        self.estadoInicial = None
        self.estadoFinal = None

        for segmento in problema['segments']:
            self.velMax = max(self.velMax, (segmento['speed'] / 3.6))
            self.interseccionAccion[segmento['origin']].append(
                (segmento['destination'], segmento['distance'] / (segmento['speed'] / 3.6))
            )

        for k in self.interseccionAccion:
            self.interseccionAccion[k] = tuple(self.interseccionAccion[k])

        self.candidatos = tuple(problema['candidates'])
        self.numeroEstaciones = problema['number_stations']

In [11]:
class Estado:
    def __init__(self, id, longitud, latitud):
        self.id = id
        self.longitud = longitud
        self.latitud = latitud

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, otro):
        return self.id == otro.id

In [12]:
class Accion:
    def __init__(self, acciones):
        self.acciones = acciones

In [13]:
class Nodo:
    def __init__(self, estado, accion, padre=None, coste=0.0):
        self.estado = estado
        self.accion = accion
        self.padre = padre
        self.coste = coste

    def __eq__(self, otro):
        return self.estado.id == otro.estado.id

    def __lt__(self, otro):
        return self.estado.id < otro.estado.id

    def __hash__(self):
        return hash(self.estado.id)

In [14]:
class Heuristica:
    @staticmethod
    def calculo_heuristica(estado1: Estado, estado2: Estado, velMax):
        dx = estado1.longitud - estado2.longitud
        dy = estado1.latitud - estado2.latitud
        distancia = math.sqrt(dx * dx + dy * dy)
        return (distancia * 1000) / velMax

In [15]:
class Busqueda:
    def __init__(self, nombre_archivo):
        self.problema = Problema(nombre_archivo)
        self.listaAbiertos = []

    def insertarNodo(self, nodo, listaNodos):
        pass

    def extraerNodo(self, listaNodos):
        pass

    def vacio(self, listaNodos):
        pass

    def nuevosNodos(self, nodo, listaAbiertos, listaExpandidos):
        contador = 0
        if nodo.accion:
            for accion in nodo.accion.acciones:
                destino = accion[0]
                cost = nodo.coste + accion[1]
                estado_nuevo = Estado(destino,
                                      self.problema.interseccionesCoordenadas[destino][0],
                                      self.problema.interseccionesCoordenadas[destino][1])
                nodoNuevo = Nodo(estado_nuevo, None, nodo, cost)
                self.insertarNodo(nodoNuevo, listaAbiertos)
                contador += 1
        return contador

    def buscar(self, inicio, destino):
        self.listaAbiertos.clear()
        coordenada_inicio = self.problema.interseccionesCoordenadas[inicio]
        coordenada_final = self.problema.interseccionesCoordenadas[destino]
        self.problema.estadoInicial = Estado(inicio, coordenada_inicio[0], coordenada_inicio[1])
        self.problema.estadoFinal = Estado(destino, coordenada_final[0], coordenada_final[1])

        acciones_iniciales = self.problema.interseccionAccion[inicio]
        accion_inicial = Accion(acciones_iniciales)
        nodoProgenitor = Nodo(self.problema.estadoInicial, accion_inicial)
        listaExpandidos = set()
        self.insertarNodo(nodoProgenitor, self.listaAbiertos)

        while not self.vacio(self.listaAbiertos):
            nodo = self.extraerNodo(self.listaAbiertos)
            if nodo.estado not in listaExpandidos:
                if nodo.estado.id == destino:
                    return nodo.coste
                listaExpandidos.add(nodo.estado)
                acciones = self.problema.interseccionAccion[nodo.estado.id]
                if acciones:
                    nodo.accion = Accion(acciones)
                    self.nuevosNodos(nodo, self.listaAbiertos, listaExpandidos)
        return 3600 * 5

In [16]:
class AEstrella(Busqueda):
    def __init__(self, nombre_archivo):
        super().__init__(nombre_archivo)

    def insertarNodo(self, nodo, listaNodos):
        f_n = Heuristica.calculo_heuristica(nodo.estado, self.problema.estadoFinal, self.problema.velMax) + nodo.coste
        heapq.heappush(listaNodos, (f_n, nodo))

    def extraerNodo(self, listaAbiertos):
        return heapq.heappop(listaAbiertos)[1]

    def vacio(self, listaNodos):
        return len(listaNodos) == 0

In [17]:
class BusquedaAleatoria:
    def __init__(self, nombre_archivo, caching=True):
        self.problema = Problema(nombre_archivo)
        self.a_estrella = AEstrella(nombre_archivo)
        self.valoresAlmacenados = {}
        self.caching = caching
        self.contador_llamadas_aestrella = 0
        self.mejoresIndividuos = []
        self.mejorTiempoMedio = 0
        self.desviacion = 0

    def a_estrella_Cache(self, origen, destino):
        ruta = (origen, destino)
        if self.caching:
            if ruta in self.valoresAlmacenados:
                tiempo = self.valoresAlmacenados[ruta]
            else:
                self.contador_llamadas_aestrella += 1
                tiempo = self.a_estrella.buscar(origen, destino)
                self.valoresAlmacenados[ruta] = tiempo
        else:
            self.contador_llamadas_aestrella += 1
            tiempo = self.a_estrella.buscar(origen, destino)
        return tiempo

    def buscar(self, semilla, numeroLoops):
        random.seed(semilla)
        mejorTiempoMedio = sys.float_info.max
        listaResultados = []

        for contador in range(numeroLoops):
            print("Vuelta numero: ", contador + 1)
            aleatorio = tuple(random.sample(self.problema.candidatos, self.problema.numeroEstaciones))
            tiempo_betha = 0.0
            poblacion_total = 0.0

            for interseccion in self.problema.candidatos:
                minimo = 3600 * 5
                origen = interseccion[0]

                peso_poblacion = interseccion[1]
                for x in aleatorio:
                    destino = x[0]
                    tiempo = self.a_estrella_Cache(origen, destino)
                    if tiempo < minimo:
                        minimo = tiempo

                tiempo_betha += peso_poblacion * minimo
                poblacion_total += peso_poblacion

            if poblacion_total > 0:
                resultado = tiempo_betha / poblacion_total
                listaResultados.append(resultado)
                if resultado < mejorTiempoMedio:
                    self.mejoresIndividuos = [x[0] for x in aleatorio]
                    mejorTiempoMedio = resultado

        if listaResultados:
            desviacion = statistics.pstdev(listaResultados)
        else:
            desviacion = 0.0

        return mejorTiempoMedio, desviacion

In [18]:
class AlgoritmoGenetico:
    def __init__(self, busqueda_aleatoria, tam_poblacion, prob_cruce, prob_mutacion, max_generaciones):
        self.ba = busqueda_aleatoria
        self.tam_poblacion = tam_poblacion
        self.prob_cruce = prob_cruce
        self.prob_mutacion = prob_mutacion
        self.max_generaciones = max_generaciones
        self.num_candidatos = len(self.ba.problema.candidatos)
        self.estaciones_requeridas = self.ba.problema.numeroEstaciones
        self.listaFitnessGeneracion = []

    def generar_individuo(self):
        indices = random.sample(range(self.num_candidatos), self.estaciones_requeridas)
        individuo = ['0'] * self.num_candidatos
        for i in indices:
            individuo[i] = '1'
        return "".join(individuo)

    def decodificar(self, individuo):
        estaciones = []
        for i, bit in enumerate(individuo):
            if bit == '1':
                estaciones.append(self.ba.problema.candidatos[i])
        return tuple(estaciones)

    def evaluar(self, individuo):
        if individuo.count('1') != self.estaciones_requeridas:
            return sys.float_info.min

        estaciones = self.decodificar(individuo)
        if len(estaciones) == 0:
            return sys.float_info.min

        tiempo_betha = 0.0
        pop_total = 0.0
        for interseccion in self.ba.problema.candidatos:
            minimo = sys.float_info.max
            origen = interseccion[0]
            peso_poblacion = interseccion[1]
            for x in estaciones:
                destino = x[0]
                tiempo = self.ba.a_estrella_Cache(origen, destino)
                if tiempo < minimo:
                    minimo = tiempo
            tiempo_betha += peso_poblacion * minimo
            pop_total += peso_poblacion

        if pop_total > 0:
            resultado = tiempo_betha / pop_total
            fitness = 1.0 / (1.0 + resultado)
        else:
            fitness = sys.float_info.min
        return fitness

    def seleccionar(self, poblacion, fitnesses, k=3):
        seleccionados = []
        for _ in range(len(poblacion)):
            aspirantes = random.sample(list(zip(poblacion, fitnesses)), k)
            aspirantes.sort(key=lambda x: x[1], reverse=True)
            seleccionados.append(aspirantes[0][0])
        return seleccionados

    def cruzar(self, p1, p2):
        if random.random() < self.prob_cruce:
            punto = random.randint(1, len(p1) - 1)
            h1 = p1[:punto] + p2[punto:]
            h2 = p2[:punto] + p1[punto:]
            return h1, h2
        else:
            return p1, p2

    def mutar(self, individuo):
        individuo = list(individuo)
        for i in range(len(individuo)):
            if random.random() < self.prob_mutacion:
                individuo[i] = '1' if individuo[i] == '0' else '0'
        return "".join(individuo)

    def algoritmo(self):
        poblacion = [self.generar_individuo() for _ in range(self.tam_poblacion)]
        lista_deFitness = [self.evaluar(individuo) for individuo in poblacion]
        mejor_fitness = max(lista_deFitness)

        for generacion in range(0, self.max_generaciones + 1):
            poblacion_selec = self.seleccionar(poblacion, lista_deFitness)
            nueva_pob = []
            for i in range(0, self.tam_poblacion, 2):
                p1 = poblacion_selec[i]
                p2 = poblacion_selec[(i + 1) % self.tam_poblacion]
                h1, h2 = self.cruzar(p1, p2)
                h1 = self.mutar(h1)
                h2 = self.mutar(h2)
                nueva_pob.append(h1)
                nueva_pob.append(h2)

            lista_deFitness = [self.evaluar(individuo) for individuo in nueva_pob]
            poblacion = nueva_pob
            mejor_fitness = max(lista_deFitness)
            desv = statistics.pstdev(lista_deFitness) if len(lista_deFitness) > 1 else 0.0

            self.listaFitnessGeneracion.append({'gen': generacion, 'mejor': mejor_fitness, 'desv': desv})

        indice_MayorValor = lista_deFitness.index(max(lista_deFitness))
        mejor = poblacion[indice_MayorValor]
        return mejor, max(lista_deFitness), self.decodificar(mejor)

In [19]:
if __name__ == "__main__":

    #toy nombre_archivo = r"C:\Users\eduar\PycharmProjects\S.-Inteligentes-proyecto\sample-problems-lab2\toy\calle_del_virrey_morcillo_albacete_250_3_candidates_15_ns_4.json"
    #small nombre_archivo = r"C:\Users\eduar\PycharmProjects\S.-Inteligentes-proyecto\sample-problems-lab2\small\calle_herreros_albacete_250_1_candidates_25_ns_5.json"
    #medio nombre_archivo=r"C:\Users\eduar\PycharmProjects\S.-Inteligentes-proyecto\sample-problems-lab2\medium\calle_franciscanos_albacete_500_3_candidates_107_ns_8.json"
    nombre_archivo = r"C:\Users\pasat\OneDrive\Documentos\InteligentesPractica2\sample-problems-lab2\toy\calle_del_virrey_morcillo_albacete_250_3_candidates_15_ns_4.json"
    num_iteraciones = 30
    resultados_aleatorio = []
    resultados_genetico = []


    for i in range(0, num_iteraciones + 1):
        random.seed(i)
        print(f"Ejecutando Búsqueda Aleatoria iteracion {i} de {num_iteraciones}")


        z = BusquedaAleatoria(nombre_archivo, caching=True)
        inicio = time.time()
        resultado = z.buscar(i, 1000)
        tiempo_final = time.time() - inicio


        resultados_aleatorio.append({
            'semilla': i,
            'mejor_tiempo_medio': resultado[0],
            'desviacion': resultado[1],
            'num_llamadas_AEstrella': z.contador_llamadas_aestrella,
            'tiempo_ejecucion': round(tiempo_final, 3),
            'mejores_individuos': z.mejoresIndividuos
        })


    ruta_aleatorio = r"C:\Users\pasat\OneDrive\Documentos\InteligentesPractica2\JsonGenetico\AlgoritmoAleatorio_Resultados.json"
    with open(ruta_aleatorio, 'w') as archivo_aleatorio:
        json.dump(resultados_aleatorio, archivo_aleatorio, indent=4)
    print(f"Datos de Búsqueda Aleatoria guardados en: {ruta_aleatorio}")


    for i in range(0, num_iteraciones + 1):
        random.seed(i)
        print(f"Ejecutando Algoritmo Genético iteracion {i} de {num_iteraciones}")


        z_ag = BusquedaAleatoria(nombre_archivo, caching=True)
        ag = AlgoritmoGenetico(z_ag, 50, 0.95, 0.01, 1000)
        inicio = time.time()
        mejor_sol, mejor_fit, estaciones_sol = ag.algoritmo()
        tiempo_finalAG = time.time() - inicio


        resultados_genetico.append({
            'semilla': i,
            'mejor_individuo': mejor_sol,
            'mejor_fitness': mejor_fit,
            'estaciones_solucion': estaciones_sol,
            'num_llamadas_AEstrella': z_ag.contador_llamadas_aestrella,
            'tiempo_ejecucion': round(tiempo_finalAG, 3)
        })


    ruta_genetico = r"C:\Users\pasat\OneDrive\Documentos\InteligentesPractica2\JsonGenetico\AlgoritmoAleatorio_Resultados.json"
    with open(ruta_genetico, 'w') as archivo_genetico:
        json.dump(resultados_genetico, archivo_genetico, indent=4)
    print(f"Datos de Algoritmo Genético guardados en: {ruta_genetico}")

Ejecutando Búsqueda Aleatoria iteracion 0 de 30
Vuelta numero:  1
Vuelta numero:  2
Vuelta numero:  3
Vuelta numero:  4
Vuelta numero:  5
Vuelta numero:  6
Vuelta numero:  7
Vuelta numero:  8
Vuelta numero:  9
Vuelta numero:  10
Vuelta numero:  11
Vuelta numero:  12
Vuelta numero:  13
Vuelta numero:  14
Vuelta numero:  15
Vuelta numero:  16
Vuelta numero:  17
Vuelta numero:  18
Vuelta numero:  19
Vuelta numero:  20
Vuelta numero:  21
Vuelta numero:  22
Vuelta numero:  23
Vuelta numero:  24
Vuelta numero:  25
Vuelta numero:  26
Vuelta numero:  27
Vuelta numero:  28
Vuelta numero:  29
Vuelta numero:  30
Vuelta numero:  31
Vuelta numero:  32
Vuelta numero:  33
Vuelta numero:  34
Vuelta numero:  35
Vuelta numero:  36
Vuelta numero:  37
Vuelta numero:  38
Vuelta numero:  39
Vuelta numero:  40
Vuelta numero:  41
Vuelta numero:  42
Vuelta numero:  43
Vuelta numero:  44
Vuelta numero:  45
Vuelta numero:  46
Vuelta numero:  47
Vuelta numero:  48
Vuelta numero:  49
Vuelta numero:  50
Vuelta nume